### start

In [3]:
# pyright: reportMissingImports=false
# pyright: reportMissingModuleSource=false

import uuid
import random
import hashlib
import os
import sys
import gc
import time
import copy
import logging
import re
from itertools import chain,combinations
import pdb
import math
from pathlib import Path
import pickle
import cloudpickle
import datetime
from datetime import datetime,timedelta
import json
import warnings
import yaml
from typing import Dict,Union,List,Any,Tuple
import pytest
import importlib
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import IPython
import requests
import pandas_gbq
from google.cloud import bigquery
from dreams_core.googlecloud import GoogleCloud as dgc
from dreams_core import core as dc
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import progressbar
import warnings
import boto3

# load_dotenv(Path("../../../Local/.env"))

# Custom format function for displaying |numbers/
pd.set_option('display.float_format', lambda x: f'{x:.12g}')
# pd.reset_option('display.float_format')

# Suppress warnings
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'
warnings.filterwarnings('ignore', message='.*frozen modules.*')
warnings.filterwarnings("ignore", message="MallocStackLogging")

# silence pygame donation request
os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"
os.environ['LOGGING_FILE'] = "../../../Local/logs/wallet_modeling.log"
os.environ['NOTIFICATION_SOUNDS_DIR'] = "../../../Local"

# Dark mode charts
plt.rcParams['figure.facecolor'] = '#181818'  # Custom background color (dark gray in this case)
plt.rcParams['axes.facecolor'] = '#181818'
plt.rcParams['text.color'] = '#afc6ba'
plt.rcParams['axes.labelcolor'] = '#afc6ba'
plt.rcParams['xtick.color'] = '#afc6ba'
plt.rcParams['ytick.color'] = '#afc6ba'
plt.rcParams['axes.titlecolor'] = '#afc6ba'
matplotlib.rcParams['text.usetex'] = False
matplotlib.rcParams['mathtext.default'] = 'regular'


# import local modules
# pyright: reportMissingImports=false
sys.path.append('..//src')

import etls.s3_pipeline as sp
import utils as u
from utils import ConfigError

# reload all modules
modules = [
    sp,
    u
]

# load all configs


# Set the custom error handler
ipython = IPython.get_ipython()
ipython.set_custom_exc((Exception,), u.notify_on_failure)

player = u.AmbientPlayer()
player.stop_all_players()

# configure logger
logger = u.setup_notebook_logger('../logs/notebook_logs.log')
logger.setLevel(logging.INFO)


# u.export_code(
#     code_directories=[
#         # 'training_data',
#         # 'base_modeling',
#         # 'wallet_features',
#         # 'wallet_insights',
#         # 'wallet_modeling',
#         # 'wallet_investing',
#         # 'coin_modeling',
#         # 'coin_investing',
#         # 'coin_insights',
#     ],
#     # include_config = True,
#     # include_config = False,
#     # ipynb_notebook = 'DDA-769 coin model score dist toggle.ipynb'
# )

[importlib.reload(module) for module in modules]
u.notify('retro')

logger.milestone("Good morning, let's get to work")

# pd.to_datetime('2025-04-30') - timedelta(days=30)

[23/Jun/25 12:56:30] MILESTONE [801714797.<module>:121] Good morning, let's get to work


# Initial ETLs

In [ ]:
sp.upload_folder_to_s3(
    '../s3_wallet_loading_queue/dda858_initial_wallet_batch',
    'wallet-training-data',
    'imports_raw'
)

[23/Jun/25 12:57:03] INFO [s3_pipeline.upload_folder_to_s3:63] Ready to upload 208 files with total size 0.84GB
[23/Jun/25 12:57:03] INFO [s3_pipeline.upload_folder_to_s3:64] Target: s3://wallet-training-data/imports_raw/dda858_initial_wallet_batch
[23/Jun/25 12:57:07] INFO [s3_pipeline.upload_folder_to_s3:82] Uploading y_train_230710.parquet -> s3://wallet-training-data/imports_raw/dda858_initial_wallet_batch/y_train_230710.parquet
[23/Jun/25 12:57:08] INFO [s3_pipeline.upload_folder_to_s3:82] Uploading x_eval_240306.parquet -> s3://wallet-training-data/imports_raw/dda858_initial_wallet_batch/x_eval_240306.parquet
[23/Jun/25 12:57:09] INFO [s3_pipeline.upload_folder_to_s3:82] Uploading y_test_230710.parquet -> s3://wallet-training-data/imports_raw/dda858_initial_wallet_batch/y_test_230710.parquet
[23/Jun/25 12:57:09] INFO [s3_pipeline.upload_folder_to_s3:82] Uploading x_eval_230908.parquet -> s3://wallet-training-data/imports_raw/dda858_initial_wallet_batch/x_eval_230908.parquet
[23/J

# Code begins